[View in Colaboratory](https://colab.research.google.com/github/hamil168/Chatbots/blob/master/Preproc.ipynb)

In [0]:
# For a fresh Colab instance, clone fresh:
!pip install -q xlrd
!git clone https://github.com/hamil168/Chatbots

Cloning into 'Chatbots'...
remote: Counting objects: 23, done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 23 (delta 8), reused 13 (delta 2), pack-reused 0
Unpacking objects: 100% (23/23), done.


In [21]:
# Change to Colab directory:
!cd Chatbots/

# For an existing Colab instance, pull from master
#!git pull https://github.com/hamil168/Chatbots master

/bin/sh: 1: cd: can't cd to Chatbots/


In [25]:
# Files as they appear in the repo clone
ls

Cornell Movie Script Database EDA.ipynb  movie_lines.txt
movie_conversations.txt                  Preproc.ipynb


In [26]:
# Start script

import numpy as np
import tensorflow as tf
import re
import time

In [27]:
lines = open('movie_lines.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')
conversations = open('movie_conversations.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')

In [32]:
print("\n".join(lines[0:10]))

L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!
L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!
L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.
L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?
L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.
L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow
L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.
L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No
L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I'm kidding.  You know how sometimes you just become this "persona"?  And you don't know how to quit?
L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?


In [33]:
print("\n".join(conversations[0:10]))

u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L363', 'L364']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L365', 'L366']


In [0]:
**TODO**: Create Dictionary to map lines to conversations
  
Goal is a dataset that contains 2 columns, input for NN, output is target for supervised learning
map each line with its ID



In [34]:
# iterate through each line, split into different elements, get key, get value
id2line = {}

for line in lines:
  _line = line.split(' +++$+++ ')
  
  if len(_line) == 5:
    id2line[_line[0]] = _line[4]
  

In [44]:
id2line['L1045']

'They do not!'